In [1]:
# from retriever import retrieve_documents
# from llm.model import call_llm
from RAG.utils.pdf_loader import load_pdf
from RAG.utils.preprocess import clean_text, chunk_text
from RAG.database.milvus_connector import connect_to_milvus
from RAG.database.vector_store import save_to_milvus

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [2]:
connect_to_milvus()

Connected to Milvus


In [3]:
# 1. PDF에서 텍스트 추출
raw_text = load_pdf('../../data/documents/genesis-g90-black-24-manual-kor.pdf')
if not raw_text:
    raise ValueError("Failed to extract text from PDF.")
raw_text

'RS4_G90_KO.book Page 1\n안전 및 차량 손상 경고\n본 취급설명서는 고객 및 차량의 안전과 관련한 심각한 위험과 제품 사용에 대한 올바른 정보\n를 사전에 알리는 안전경고 표시입니다. 지시 사항은 반드시 숙지하여 지켜주십시오.\n경고, 주의 표시 경고, 주의가 있는 문장 및 진하게 표시되어 있는 부분은 특히 유\n념하십시오.\n(cid:1155)(cid:1162)\n사람이 다치거나 사망의 우려가 있는 경우의 경고 표시입니다.\n(cid:2737)(cid:2628)\n차량이 고장나거나 손상될 우려가 있는 경우의 주의 표시입니다.\n(cid:2444)(cid:2439)(cid:1601)(cid:1245)\n차량 용어 또는 추가 설명이 필요한 정보 표시입니다.\n안전을 위해 반드시 지켜야 하는 금지 표시입니다.\n선택 또는 미장착 본 취급설명서는 모든 트림 모델 및 선택 사양을 포함하여 설명하\n사양 표시 고 있습니다.\n따라서 고객님 차량에 장착되지 않은 사양이 설명 될 수 있습니다.\nRS4_G90_KO.book Page 2\n내용 찾기 방법 설명\n내용으로 찾을 때\n목차를 활용하세요.\n명칭을 모를 때\n그림 목차 (내관도 I) 를 활용하세요.\n명칭으로 찾을 때\n색인(명칭) 목차를 활용하세요.\nRS4_G90_KO.book Page 3\n제작결함 안내 자동차제작자 등(부품제작자 등): 현대자동차(주)\n주 소: 서울특별시 서초구 헌릉로 12(양재동)\n연락처: 080-700-6000\n제작결함 안내 귀하의 자동차 또는 자동차부품에 잦은 고장 등의 문제\n로 교통사고를 유발할 수 있는 결함이 있다고 판단되\n(제50조 관련)\n면, 자기 및 다른 사람의 안전을 위하여 즉시 현대자동\n차(주)와 제작결함조사를 시행하는 한국교통안전공단\n자동차안전연구원에 연락하여 주시기 바랍니다.\n한국교통안전공단 자동차안전연구원은 소비자 불만사\n항 등을 접수하여 분석한 후 해당 자동차 또는 자동차\n부품에 제작결함의 가능성이 있다고 판단되는 경우 제\n

In [4]:
# 2. 텍스트 전처리
processed_text = clean_text(raw_text)

In [6]:
# 3. 텍스트 청킹
chunks = chunk_text(processed_text, chunk_size=100, overlap=20)

In [12]:
len(max(chunks))

407

In [7]:
# 4. 임베딩 생성 및 Milvus 저장
document_id = 'genesis-g90-black-24-manual-kor.pdf'


In [23]:
from RAG.database.milvus_connector import get_collection
from RAG.llm.embedding import generate_embedding

collection = get_collection('manual')

Connected to Milvus
컬렉션 가져오는중


In [9]:
embedding = generate_embedding(chunks)



pre tokenize: 100%|██████████| 99/99 [00:00<00:00, 841.30it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 99/99 [00:03<00:00, 26.47it/s]


Embeddings: {'dense': [array([-0.04123 ,  0.01482 , -0.05963 , ..., -0.010895,  0.03806 ,
        0.007454], dtype=float16), array([-0.0748   , -0.015594 , -0.02773  , ..., -0.003378 ,  0.01797  ,
        0.0006356], dtype=float16), array([-0.05292 , -0.01179 , -0.002193, ..., -0.02658 , -0.01182 ,
        0.001774], dtype=float16), array([ 0.006844, -0.004353, -0.05338 , ...,  0.01299 ,  0.01197 ,
       -0.000968], dtype=float16), array([-0.026    ,  0.005753 , -0.03494  , ...,  0.0006146,  0.0338   ,
        0.001295 ], dtype=float16), array([-0.05966 ,  0.02107 , -0.02461 , ...,  0.004524,  0.01003 ,
       -0.00608 ], dtype=float16), array([-0.03372 , -0.02132 , -0.02097 , ..., -0.0042  ,  0.002625,
       -0.002941], dtype=float16), array([-0.01802  , -0.013756 , -0.04166  , ...,  0.02551  , -0.006733 ,
        0.0006633], dtype=float16), array([-0.01752 ,  0.01024 , -0.04562 , ...,  0.010864,  0.01851 ,
       -0.03616 ], dtype=float16), array([-0.03632 ,  0.01598 , -0.0465  , .

In [10]:
embedding_dense = embedding['dense']
embedding_sparse = embedding['sparse']

In [22]:
# 컬렉션 생성
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)
dense_dim = 1024
connections.connect(host="192.168.0.130", port=19530, db_name='tailorlink')

fields = [
    # Use auto generated id as primary key
    FieldSchema(
        name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=8192),
    # Milvus now supports both sparse and dense vectors,
    # we can store each in a separate field to conduct hybrid search on both vectors
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
]
schema = CollectionSchema(fields, enable_dynamic_field=True)

col_name = "manual"
if utility.has_collection(col_name):
    Collection(col_name).drop()
col = Collection(col_name, schema, consistency_level="Strong")

sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
col.create_index("dense_vector", dense_index)
col.load()


In [24]:
import numpy as np
for i in range(0, len(chunks), 50):
    batched_entities = [
        chunks[i: i + 50],
        embedding["sparse"][i: i + 50],
        np.array(embedding["dense"][i: i + 50]).astype(np.float32),  # 리스트를 NumPy 배열로 변환한 다음 astype 적용
    ]
    collection.insert(batched_entities)
print("Number of entities inserted:", collection.num_entities)


Number of entities inserted: 0


In [21]:
max_length = max(len(chunk) for chunk in chunks)
max_length

3804